## Project 1: Sieve of Eratosthenes Algo

In [3]:
import math

def sieve_of_eratosthenes(limit):
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False

    for num in range(2, int(math.sqrt(limit)) + 1):
        if sieve[num]:
            for multiple in range(num * num, limit + 1, num):
                sieve[multiple] = False

    return [index for index, is_prime in enumerate(sieve) if is_prime]

# Constants
n = 8201997
N = 5000000 + n


estimated_limit = int(N * math.log(N) * 1.2)

# Find the N-th prime
primes = sieve_of_eratosthenes(estimated_limit)

if len(primes) >= N:
    print("The (5000000 + n)-th prime number is:", primes[N - 1])
else:
    print("Error: Not enough primes generated. Increase the limit.")

# Count primes between 2^25 and 2^26
lower_bound = 2**25
upper_bound = 2**26
primes_in_range = [p for p in primes if lower_bound <= p <= upper_bound]

print("Number of primes between 2^25 and 2^26:", len(primes_in_range))

The (5000000 + n)-th prime number is: 240776881
Number of primes between 2^25 and 2^26: 1894120


# Project 2 : ElGamal Cryptosystem Decryption using Pohlig-Hellman Algorithm


we implement the decryption of a message encrypted using the ElGamal cryptosystem. The decryption process involves using the Pohlig-Hellman algorithm to find the private key of Bob (denoted `x_B`), and then decrypting the ciphertext using this key.

The ciphertext is a list of pairs of numbers, which were encrypted using Alice's public key, and we use the following parameters:

- **p**: A large prime number defining the group.
- **g**: A generator of the cyclic group.
- **gB**: Bob's public key.
- **aB**: Bob's private key (which we need to recover).
- **ciphertext**: The encrypted message consisting of pairs of numbers.

We will:
1. Compute the private key `x_B` using the Pohlig-Hellman algorithm.
2. Decrypt the ciphertext using the private key.
3. Translate the decrypted message into readable text.

The output will include the decrypted message and its translation into characters corresponding to the message format.

### Extended Euclidean Algorithm

In [4]:
# Extended GCD algorithm
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

# Modular inverse using Extended GCD
def modular_inverse(a, p):
    gcd, x, _ = extended_gcd(a, p)
    if gcd != 1:
        raise ValueError("Modular inverse does not exist")
    return x % p

## Prime factorization

In [5]:
# Prime factorization of p-1 (without using sympy)
def prime_factors(n):
    factors = []
    # Check for number of 2s
    while n % 2 == 0:
        factors.append(2)
        n //= 2
    # Check for odd numbers
    for i in range(3, int(n**0.5) + 1, 2):
        while n % i == 0:
            factors.append(i)
            n //= i
    if n > 2:
        factors.append(n)
    return factors

## Chinese Remainder Theorem

In [6]:
# Chinese Remainder Theorem
def chinese_remainders(residues, moduli):
    total = 0
    prod = 1
    for mod in moduli:
        prod *= mod
    for res, mod in zip(residues, moduli):
        p = prod // mod
        total += res * modular_inverse(p, mod) * p
    return total % prod

#### Pohlig-Hellman Algorithm
This algorithm is used to compute discrete logarithms in groups whose order has small prime factors.

In [7]:
# Pohlig-Hellman algorithm
def pohlig_hellman(g, h, p, q):
    n = p - 1
    factors = prime_factors(n)
    residues = []
    moduli = []
    for q in factors:
        e = n // q
        gq = pow(g, e, p)
        hq = pow(h, e, p)
        for i in range(q):
            if pow(gq, i, p) == hq:
                residues.append(i)
                moduli.append(q)
                break
    return chinese_remainders(residues, moduli)

## Decryption

In [8]:
# Decrypting the message
def decrypt_message(ciphertext, p, g, gB):
    # Compute private key x_B using Pohlig-Hellman
    x_B = pohlig_hellman(g, gB, p, p-1)  # Assuming q = p - 1 for simplicity

    decrypted_message = []
    for c1, c2 in ciphertext:
        # Calculate s = c1^x_B mod p
        s = pow(c1, x_B, p)
        # Calculate the modular inverse of s
        s_inv = modular_inverse(s, p)
        # Decrypt the message m
        m = (c2 * s_inv) % p
        decrypted_message.append(m)

    return decrypted_message, x_B

# Mapping from number to character
def translate_message(x):
    """
    Convert a decrypted number into its corresponding character(s).
    Each number is split into two-digit pairs and mapped to characters.
    """
    # Define the mapping for numbers to characters
    let = {i: chr(64 + i - 10) for i in range(11, 37)}  # Maps 11-36 to A-Z
    spe = {41: ' ', 42: '"', 43: '.', 44: ',', 45: '?'}  # Special characters
    mapping = {**let, **spe}  # Combine letter and special characters

    # Convert the decrypted number into a string and map it
    return ''.join(mapping.get(int(str(x)[i:i+2]), '?') for i in range(0, len(str(x)), 2))

#### Full Decryption Process
Now, we can combine everything. First, we will compute Bob's private key using the Pohlig-Hellman algorithm. Then, we'll decrypt each ciphertext pair using the elgamal_decrypt function and finally decode the message.

In [7]:
ciphertext = [
    [83025882561049910713, 66740266984208729661],
    [117087132399404660932, 44242256035307267278],
    [67508282043396028407, 77559274822593376192],
    [60938739831689454113, 14528504156719159785],
    [5059840044561914427, 59498668430421643612],
    [92232942954165956522, 105988641027327945219],
    [97102226574752360229, 46166643538418294423]
]

p = 123456789987654353003
g = 123456789
gB = 39318628345168608817

# Decrypt the message
decrypted_message, x_B = decrypt_message(ciphertext, p, g, gB)

translated_message = [translate_message(m) for m in decrypted_message]

# Output results
print(f"Computed private key x_B: {x_B}")
print(f"Decrypted Message: {decrypted_message}")
print(f"Translated Message: {''.join(translated_message)}")

Computed private key x_B: 5191
Decrypted Message: [19244117112225192941, 16191522142944411631, 22224125164116222533, 15282944412628192319, 30193215411522152315, 24302941141124131541, 16252841182531282943]
Translated Message: IN GALOIS FIELDS, FULL OF FLOWERS, PRIMITIVE ELEMENTS DANCE FOR HOURS.


# Overall Output

### Computed private key:
> **x_B:** 5191  

### Decrypted msg:
> [19244117112225192941, 16191522142944411631, 22224125164116222533, 15282944412628192319,  
30193215411522152315, 24302941141124131541, 16252841182531282943]

### Translated msg:

> IN GALOIS FIELDS, FULL OF FLOWERS, PRIMITIVE ELEMENTS DANCE FOR HOURS.


# Project 3: Pohlig-Hellman, Miller-Rabin

In [1]:
import random

# Function to check primality using Miller-Rabin test
def is_prime(n, k=10):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False

    # Write n-1 as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # Miller-Rabin test
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

# Generate a large prime number of approximately 'digits' length
def generate_large_prime(digits):
    while True:
        num = random.randint(10**(digits-1), 10**digits - 1)
        if is_prime(num):
            return num

# Generate p1 and p2 following the given constraints
def generate_p1_p2():
    while True:
        p1 = generate_large_prime(233)  # A prime of around 233 digits
        p2 = generate_large_prime(466)  # A prime of around 466 digits
        if p1 < p2 < p1**3:
            return p1, p2

# Generate p = 2 * p1 * p2 + 1
while True:
    p1, p2 = generate_p1_p2()
    p = 2 * p1 * p2 + 1
    if len(str(p)) >= 700 and is_prime(p):
        break


print("Generated values:")
print(f"p1: {p1} (length: {len(str(p1))})")
print(f"p2: {p2} (length: {len(str(p2))})")
print(f"p: {p} (length: {len(str(p))})")

# Generate ElGamal keys
g = random.randint(2, p - 2)
sk = random.randint(2, p - 2)  # Private key
pk = pow(g, sk, p)  # Public key

print("\nElGamal Key Generation:")
print(f"Public key (g, p, pk): ({g}, {p}, {pk})")
print(f"Private key (sk): {sk}")

Generated values:
p1: 76964654513346247570515381303552648383540262237250374326580388580880487598620513273116913285652477202799863244127966902952232786198192667729722619072588083143067094369274666823847559312736744518599578814678488112625544479334283880131 (length: 233)
p2: 8635430222160857351730635918352285858270423559152758664185858008490216415435585329085177194045442834264438925840030658424494161291713078650838775459507149069647247859266481004711577462334483494212209964514611217826573503061264310337718982346959351037182054068246101893076564492070619018335003146215286512431142028876595526762751014223026562762002508630453933677648028132723526374757453521057128289642424075120206396102107792318218226151085000722065447255717695427321 (length: 466)
p: 13292458072454384382452056247711214479557340404322602216479033724240703231485396351231626348490206688559374521162154695455472710937045822873710824337210654662576136363154721895456186222136242548430531707826638196201901678651771565636491341638

In [2]:
# ElGamal Encryption
def elgamal_encrypt(message, g, p, pk):
    m = int.from_bytes(message.encode(), 'big')
    if m >= p:
        raise ValueError("Message too large!")
    k = random.randint(2, p - 2)
    c1 = pow(g, k, p)
    c2 = (m * pow(pk, k, p)) % p
    return c1, c2

# ElGamal Decryption
def elgamal_decrypt(c1, c2, sk, p):
    s = pow(c1, sk, p)
    m = (c2 * pow(s, -1, p)) % p
    return m.to_bytes((m.bit_length() + 7) // 8, 'big').decode()

# Example Message Encryption and Decryption
message = "African Institute for Mathematical Sciences"
c1, c2 = elgamal_encrypt(message, g, p, pk)
decrypted_message = elgamal_decrypt(c1, c2, sk, p)

print("\nElGamal Encryption & Decryption:")
print(f"Original Message: {message}")
print(f"Ciphertext: (c1={c1}, c2={c2})")
print(f"Decrypted Message: {decrypted_message}")


ElGamal Encryption & Decryption:
Original Message: African Institute for Mathematical Sciences
Ciphertext: (c1=866599083354253750219684547593643050378144909425196613356608676321901692418653632510648369117112310533465599251417428945400079865434006208330457544153060497866026051314442537625388205596115581269770255362647172909581056953696970382156307522740385541647343602798879650626734076569117937473300165822340290774588980225031566515285059968430758277683140743436125026453821250841442595260683580278028826042566760252630794591639611798866079695253825958059529398151789699124449045159956405948663368447563317804593548661295527952875579805069744191091866200055670965388403737018036178111606835599074757249865296865320102938345333411081618239137237123886602657222259983688326839646704150099261962164697684568808, c2=3520031454158161906118148980557093249379808917293318934781534706050344704762159769485537701169780423182868036473946019257170913343597382056465269938013549396477924491137957765433770387

# Overall ouput of project 3:

# Prime Number Generation  
This section generates the prime numbers \( p_1 \) and \( p_2 \), then computes \( p \) ensuring it meets the conditions.  

## Generated Values:
- **\( p_1 \)**:  
  76964654513346247570515381303552648383540262237250374326580388580880487598620513273116913285652477202799863244127966902952232786198192667729722619072588083143067094369274666823847559312736744518599578814678488112625544479334283880131  
  _(length: 233 digits)_

- **\( p_2 \)**:  
  8635430222160857351730635918352285858270423559152758664185858008490216415435585329085177194045442834264438925840030658424494161291713078650838775459507149069647247859266481004711577462334483494212209964514611217826573503061264310337718982346959351037182054068246101893076564492070619018335003146215286512431142028876595526762751014223026562762002508630453933677648028132723526374757453521057128289642424075120206396102107792318218226151085000722065447255717695427321  
  _(length: 466 digits)_

- **\( p \) (Computed as \( p = 2 \times p_1 \times p_2 + 1 \))**:  
  1329245807245438438245205624771121447955734040432260221647903372424070323148539635123162634849020668855937452116215469545547271093704582287371082433721065466257613636315472189545618622213624254843053170782663819620190167865177156563649134163836126393924486381733752388468702770998744363504469730556328525299032739164713076807403767911277232841442534394704736909017079839740971564581861324868566341899695769488017882031559382060861954633478228747824060628770380837470714698264459806675537815394186251341475771064377058619298818994969248014474815290096123835932412374734914507212192528387072528144395132916223295475012582357094248467257210932274503563897905895919261274429876964566868626722279972918103  
  _(length: 700 digits)_


> # ElGamal Encryption & Decryption  

In this step, we encrypt and decrypt a message using the generated ElGamal key pair.  

## Original Message:  
**African Institute for Mathematical Sciences**  

## Ciphertext:  
- **\( c_1 \)**:  
  866599083354253750219684547593643050378144909425196613356608676321901692418653632510648369117112310533465599251417428945400079865434006208330457544153060497866026051314442537625388205596115581269770255362647172909581056953696970382156307522740385541647343602798879650626734076569117937473300165822340290774588980225031566515285059968430758277683140743436125026453821250841442595260683580278028826042566760252630794591639611798866079695253825958059529398151789699124449045159956405948663368447563317804593548661295527952875579805069744191091866200055670965388403737018036178111606835599074757249865296865320102938345333411081618239137237123886602657222259983688326839646704150099261962164697684568808  

- **\( c_2 \)**:  
  35200314541581619061181489805570932493798089172933189347815347060503447047621597694855377011697804231828680364739460192571709133435973820564652699380135493964779244911379577654337703874946090388497523474655277141290761424025201519190238732515236799750093634488265961169932886104538803428038729145237052577779726392611589039129250271718121854054589776477886736115997748583345496876789422903683857492078398547706335363644852529899829653220100358320591918164154265792718699230730389025492473737215100620702195908563204270420299687294312135282647506482366426926389519850343040385339434509586029474004018145158444985744652740151171531031388161783954429729100723050771379739050325950065702475635107815683  

## Decrypted Message:  
**African Institute for Mathematical Sciences**  
